# Preparación de datos

Limpieza del dataset AI4I 2020, creación de features derivadas y exportación a `data/ai4i_clean.csv` para su uso en la app y BentoML.

## Dataset y objetivo

Sensores de temperatura, velocidad, torque y desgaste en una fresadora. Targets binario (`Machine failure`) y multiclase (`TWF`, `HDF`, `PWF`, `OSF`, `RNF`).

## Carga y chequeos iniciales

In [ ]:
import pandas as pd
from pathlib import Path

raw_path = Path('../data/ai4i2020.csv')
if not raw_path.exists():
    raise FileNotFoundError('No se encontró ../data/ai4i2020.csv. Coloca el CSV original en data/ y vuelve a ejecutar.')

df_raw = pd.read_csv(raw_path)
df_raw.head()

In [ ]:
print('Filas, columnas:', df_raw.shape)
print('Duplicados:', df_raw.duplicated().sum())
df_raw.dtypes

In [ ]:
# Nulos por columna (el dataset original no tiene, pero lo verificamos)
df_raw.isna().sum()

## Limpieza: duplicados y nulos

- Eliminamos duplicados.
- Imputamos numéricos con mediana por si aparecen nulos en futuras ingestas.

In [ ]:
from sklearn.impute import SimpleImputer

df = df_raw.drop_duplicates().copy()
numeric_cols = df.select_dtypes(include='number').columns
imputer = SimpleImputer(strategy='median')
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])
df.head()

## Ingeniería de características

Reutilizamos la lógica de `src.features`: diferencia de temperaturas, torque normalizado por rpm y estado de desgaste discreto.

In [ ]:
from src.features import add_derived_features, RAW_NUMERIC_COLS, RAW_CATEGORICAL_COLS

missing = sorted((set(RAW_NUMERIC_COLS) | set(RAW_CATEGORICAL_COLS)) - set(df.columns))
if missing:
    raise ValueError(f'Faltan columnas requeridas: {missing}')
df = add_derived_features(df)
df.head()

In [ ]:
print('Nulos después del preprocesado:')
print(df.isna().sum())

## Guardar dataset limpio

Se almacena en `data/ai4i_clean.csv` para notebooks, Streamlit y BentoML.

In [ ]:
clean_path = Path('../data/ai4i_clean.csv')
clean_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(clean_path, index=False)
print('Guardado', clean_path.resolve())
df.head()